<a href="https://colab.research.google.com/github/hongwon2/BigQuery-Practice/blob/main/%EB%A1%9C%EA%B7%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A7%91%EA%B3%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
%load_ext google.colab.data_table

In [3]:
%%bigquery --project sql-recipe2
# 날짜별 접근 데이터를 집계하는 쿼리
select
  substr(stamp,1,10) as dt, #날짜 추출
  count(distinct long_session) as access_users, #쿠키추출
  count(distinct short_session) as access_count, #방문횟수 계산하기
  count(*) as page_view, #페이지뷰 계산하기
  1.0 * count(*) / nullif(count(distinct long_session), 0) as pv_per_user #1인당 페이지뷰
from
  `sql-recipe2.ch5.access_log`
group by
  dt
order by
  dt


,dt,access_users,access_count,page_view,pv_per_user
0,2016-10-01,4,5,8,2.000000
1,2016-10-02,4,5,7,1.750000
2,2016-10-03,3,3,4,1.333333


In [4]:
%%bigquery --project sql-recipe2
# url 별로 집계하는 쿼리
select
  url,
  count(distinct short_session) as access_count,
  count(distinct long_session) as access_users,
  count(*) as page_veiw
from
  ch5. access_log
group by
  url

,url,access_count,access_users,page_veiw
0,http://www.example.com/detail?id=2,2,2,2
1,http://www.example.com/,3,3,3
2,http://www.example.com/?utm_source=google&utm_...,1,1,1
3,http://www.example.com/detail?id=1,2,2,2
4,http://www.example.com/list/cd,3,3,3
5,http://www.example.com/detail?id=3,1,1,1
6,http://www.example.com/list/newly,3,2,3
7,http://www.example.com/list/dvd,2,2,2
8,http://www.example.com/list/dvd?utm_source=yah...,1,1,1
9,http://www.example.com/?utm_source=mynavi&utm_...,1,1,1


In [5]:
%%bigquery --project sql-recipe2
#  경로별로 집계하기
with
access_log_with_path as (
  select *,
  regexp_extract(url,'//[^/]+([^?#]+)') as url_path
from ch5.access_log
)
select
  url_path,
  count(distinct short_session) as access_count,
  count(distinct long_session) as access_users,
  count(*) as page_view
from
  access_log_with_path
group by
  url_path

  #/detail 이후는 디테일로 묶여있음
  #/list/cd ,list/dvd 같이 리스트는 카테고리 별로 나누어졏있음

,url_path,access_count,access_users,page_view
0,/detail,5,3,5
1,/,5,4,5
2,/list/cd,3,3,3
3,/list/newly,3,2,3
4,/list/dvd,3,2,3


In [6]:
%%bigquery --project sql-recipe2
#url에 의미를 부여하여집계하기
with
access_log_with_path as (
  select *,
  regexp_extract(url,'//[^/]+([^?#]+)') as url_path
from ch5.access_log
)
,
access_log_with_split_path as (
  -- 경로의 첫번째 용소아 두번째 요소 추출하기
  select *,
    split(url_path, '/')[safe_ordinal(2)]as path1,
    split(url_path, '/')[safe_ordinal(3)]as path2
  from access_log_with_path
)
,
access_log_with_page_name as (
  --경로를 슬래시로 분할하고 조건에 따라 이름 붙이기
  select *,
    case
      when path1 = 'list' then
        case
          when path2 = 'newly' then 'newly_list'
          else 'category_list'
        end
      else url_path
    end as page_name
  from access_log_with_split_path
)
select
  page_name,
  count(distinct short_session) as access_count,
  count(distinct long_session) as access_users,
  count(*) as page_view
from access_log_with_page_name
group by page_name
order by page_name

,page_name,access_count,access_users,page_view
0,/,5,4,5
1,/detail,5,3,5
2,category_list,6,4,6
3,newly_list,3,2,3
